In [1]:
!pip install google-generativeai gradio

In [5]:
!pip install --upgrade google-generativeai gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 25.1 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.4
    Uninstalling gradio_client-1.10.4:
      Successfully uninstalled gradio_client-1.10.4
  Attempting uninstall: gradio
    Found existing installation: gradio 5.37.0
    Uninstalling gradio-5.37.0:
      Successfully uninstalled gradio-5.37.0


In [6]:
import google.generativeai as genai
import gradio as gr
import smtplib
from email.message import EmailMessage

In [7]:
GEMINI_API_KEY = "your-gemini-api-key"

In [8]:
SENDER_EMAIL = "youremail@gmail.com"        # ← Your Gmail
SENDER_PASSWORD = "your-app-password"       # ← Gmail app password (not your real password)

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587

In [9]:
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-pro")

In [10]:
def generate_email(prompt, recipient, subject):
    full_prompt = f"""
Write a professional, polite email to {recipient} with subject "{subject}".
Here is what the email should say:

{prompt}
"""
    try:
        response = model.generate_content(full_prompt)
        return response.text.strip()
    except Exception as e:
        return f"❌ Gemini API Error: {str(e)}"


In [11]:
def send_email(recipient, subject, body):
    try:
        msg = EmailMessage()
        msg["From"] = SENDER_EMAIL
        msg["To"] = recipient
        msg["Subject"] = subject
        msg.set_content(body)

        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.starttls()
            smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
            smtp.send_message(msg)

        return "✅ Email sent successfully."
    except Exception as e:
        return f"❌ Failed to send email: {str(e)}"

In [12]:
def email_agent(prompt, recipient, subject):
    email_text = generate_email(prompt, recipient, subject)
    if email_text.startswith("❌"):
        return email_text, "❌ Generation Failed"

    result = send_email(recipient, subject, email_text)
    return email_text, result

iface = gr.Interface(
    fn=email_agent,
    inputs=[
        gr.Textbox(label="Prompt (What should the email say?)"),
        gr.Textbox(label="Recipient Email", placeholder="recipient@example.com"),
        gr.Textbox(label="Subject", placeholder="Subject goes here")
    ],
    outputs=[
        gr.Textbox(label="Generated Email"),
        gr.Textbox(label="Status")
    ],
    title="📧 Email Automation Agent (Gemini + Gradio)",
    description="Enter your intent and recipient. Gemini will draft and send the email."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://797543f236486c691c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
